# Set device to GPU if available

In [1]:
import torch
device = "cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu"
print(f"Using device: {device}")

Using device: cuda


# Load Dataset

In [2]:
dataset_path = f"dataset/Taskent"

from image_dataset import *

dataset = PetroTrainTestSplitDataset(folder_path=dataset_path)

train_dataset = dataset['train']
test_dataset = dataset['test']

print(len(train_dataset))
print(len(test_dataset))

print(train_dataset[0])
print(test_dataset[0])

__init__ PetroSubImageDataset: dataset/Taskent
         , image_indices=None
         , sub_image_size=480
__init__ BaseSubImageDataset with: dataset/Taskent/img
         , image_indices=None
         , sub_image_size=480
         , mask=False
__init__ BaseSubImageDataset with: dataset/Taskent/masks_machine
         , image_indices=None
         , sub_image_size=480
         , mask=True


/home/ubuntu/cs231n-project/image_dataset.py:211: UserWarning: The given NumPy array is not writable, and PyTorch does not support non-writable tensors. This means writing to this tensor will result in undefined behavior. You may want to copy the array to protect its data or make it writable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at /pytorch/torch/csrc/utils/tensor_numpy.cpp:203.)
  images = torch.stack([torch.from_numpy(img) for img in images])


756
252
tensor([[[125, 132, 140,  ..., 132, 115, 113],
         [133, 135, 131,  ..., 129, 115, 113],
         [122, 122, 120,  ..., 134, 123, 115],
         ...,
         [134, 136, 126,  ..., 148, 148, 151],
         [133, 128, 121,  ..., 158, 157, 162],
         [136, 137, 139,  ..., 174, 171, 169]],

        [[125, 132, 140,  ..., 132, 115, 113],
         [133, 135, 131,  ..., 129, 115, 113],
         [122, 122, 120,  ..., 134, 123, 115],
         ...,
         [134, 136, 126,  ..., 148, 148, 151],
         [133, 128, 121,  ..., 158, 157, 162],
         [136, 137, 139,  ..., 174, 171, 169]],

        [[125, 132, 140,  ..., 132, 115, 113],
         [133, 135, 131,  ..., 129, 115, 113],
         [122, 122, 120,  ..., 134, 123, 115],
         ...,
         [134, 136, 126,  ..., 148, 148, 151],
         [133, 128, 121,  ..., 158, 157, 162],
         [136, 137, 139,  ..., 174, 171, 169]],

        [[  2,   2,   2,  ...,   2,   2,   2],
         [  2,   2,   2,  ...,   2,   2,   2],
    

Load DINO Model and Helper Functions

In [3]:
from torch.utils.data import DataLoader
from clip_dino import DINOSegmentation, compute_iou
from dino_model import DINOPatchClassifier
from tqdm import tqdm
from einops import rearrange
import numpy as np

In [4]:
# Load smallest dino model. ViT-S/8. Here ViT-S has ~22M parameters and
# works on 8x8 patches.
dino_model = torch.hub.load('facebookresearch/dino:main', 'dino_vits8')
dino_model.eval().to(device)

Using cache found in /home/ubuntu/.cache/torch/hub/facebookresearch_dino_main


VisionTransformer(
  (patch_embed): PatchEmbed(
    (proj): Conv2d(3, 384, kernel_size=(8, 8), stride=(8, 8))
  )
  (pos_drop): Dropout(p=0.0, inplace=False)
  (blocks): ModuleList(
    (0-11): 12 x Block(
      (norm1): LayerNorm((384,), eps=1e-06, elementwise_affine=True)
      (attn): Attention(
        (qkv): Linear(in_features=384, out_features=1152, bias=True)
        (attn_drop): Dropout(p=0.0, inplace=False)
        (proj): Linear(in_features=384, out_features=384, bias=True)
        (proj_drop): Dropout(p=0.0, inplace=False)
      )
      (drop_path): Identity()
      (norm2): LayerNorm((384,), eps=1e-06, elementwise_affine=True)
      (mlp): Mlp(
        (fc1): Linear(in_features=384, out_features=1536, bias=True)
        (act): GELU(approximate='none')
        (fc2): Linear(in_features=1536, out_features=384, bias=True)
        (drop): Dropout(p=0.0, inplace=False)
      )
    )
  )
  (norm): LayerNorm((384,), eps=1e-06, elementwise_affine=True)
  (head): Identity()
)

In [5]:
from torchvision import transforms as T

transform = T.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))

# transform = T.Compose([
#     # T.Resize((480, 480)),
#     T.ToTensor(),
#     T.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
# ])


In [6]:
def get_dino_tokens_batch(X_batch):

    # X_batch.shape = [batch_size,480,480,3]

    X_batch = X_batch.float() / 255.0 # normalize
    # X_batch = X_batch.permute(0,3,1,2) # [N,H,W,C] -> [N,C,H,W]
    X_transform = torch.stack([
        transform(x) for x in X_batch
    ])
    X_transform = X_transform.to(device)

    w, h = X_transform.shape[2:]
    # 480,480

    with torch.no_grad():
        attn = dino_model.get_last_selfattention(X_transform)[:, :, 0, 1:]
        # (N,6,3600)
        # print(f"attn.shape={attn.shape}")
        nh, tokens = attn.shape[1:]
        w_feat, h_feat = w // 8, h // 8
        attn = attn.reshape(-1, nh, w_feat, h_feat)
        attn = torch.nn.functional.interpolate(attn, scale_factor=8, mode="nearest").cpu().numpy()
        all_tokens = dino_model.get_intermediate_layers(X_transform, n=1)[0]  # (N, 1+pixels, D)
    return all_tokens.cpu()

In [7]:
import numpy as np
import torch
from einops import rearrange
from scipy.stats import mode

def get_patchwise_mode(Y_batch):

    if isinstance(Y_batch, torch.Tensor):
        Y_batch = Y_batch.cpu().numpy()

    # Rearrange to 8x8 patches
    patches = rearrange(Y_batch, 'n (h ph) (w pw) -> n h w (ph pw)', ph=8, pw=8)
    # shape: (N, H//8, W//8, 64)

    # Compute mode along last axis
    patch_modes = mode(patches, axis=-1).mode  # shape: (N, H//8, W//8)

    return torch.tensor(patch_modes, dtype=torch.uint8)

# WandB Login

In [8]:
from dotenv import load_dotenv
load_dotenv()

import os

os.environ["WANDB_API_KEY"] = os.getenv("WANDB_API_KEY")

import wandb
wandb.login()

wandb: Currently logged in as: thisisrmak (thisisrmak-stanford) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

# Run train/test loop

In [9]:
wandb.init(
    project="dinov2-biotic-nb-10",
    # name="run-patchwise-mode",  # Optionally change per experiment
    config={
        "num_classes": 18,
        "batch_size": 64,
        "num_iters": 50,
        "hidden_dim": 768,
        "lr": 1e-3
    }
)

config = wandb.config

train_dataloader = DataLoader(train_dataset, batch_size=config.batch_size)
test_dataloader = DataLoader(test_dataset, batch_size=config.batch_size)

model = DINOPatchClassifier(num_classes=config.num_classes).to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=config.lr)
loss_fn = torch.nn.CrossEntropyLoss()

for iter in range(config.num_iters):

    ### TRAINING ###

    model.train()
    total_train_loss = 0.0
    correct_train = 0
    total_train = 0
    total_train_iou = 0.0

    pbar = tqdm(train_dataloader, desc=f"Train Epoch {iter+1}/{config.num_iters}")

    for batch in pbar:
        X_batch = batch[:, :-1]
        Y_batch = batch[:, -1]

        Y = get_patchwise_mode(Y_batch)
        Y = rearrange(Y, 'n h w -> (n h w)')
        Y = Y.to(device)

        with torch.no_grad():
            X = get_dino_tokens_batch(X_batch)  # [N, 3601, D=384]
            X = X[:, 1:, :]  # drop CLS token
            X = rearrange(X, 'n p d -> (n p) d')
            X = X.to(device)

        optimizer.zero_grad()
        logits = model(X)
        loss = loss_fn(logits, Y)
        loss.backward()
        optimizer.step()

        # Stats
        batch_loss = loss.item()
        total_train_loss += batch_loss

        preds = torch.argmax(logits, dim=1)
        correct_train += (preds == Y).sum().item()
        total_train += Y.size(0)

        batch_iou = compute_iou(preds.cpu().numpy(), Y.cpu().numpy(), config.num_classes)
        total_train_iou += batch_iou

        pbar.set_postfix(loss=f"{batch_loss:.2f}", iou=f"{batch_iou:.2f}")

    avg_train_loss = total_train_loss / len(train_dataloader)
    avg_train_accuracy = correct_train / total_train
    avg_train_iou = total_train_iou / len(train_dataloader)



    # print(f"Epoch {iter+1}: Train Loss={avg_train_loss:.4f}, Accuracy={avg_train_accuracy:.4f}, IoU={avg_train_iou:.4f}")

     ### VALIDATION ###

    model.eval()
    total_val_loss = 0.0
    correct_val = 0
    total_val = 0
    total_val_iou = 0.0

    with torch.no_grad():
        pbar = tqdm(test_dataloader, desc=f"Val Epoch {iter+1}/{config.num_iters}")

        for batch in pbar:
            X_batch = batch[:, :-1]
            Y_batch = batch[:, -1]

            Y = get_patchwise_mode(Y_batch)
            Y = rearrange(Y, 'n h w -> (n h w)').to(device)

            X = get_dino_tokens_batch(X_batch)
            X = X[:, 1:, :]
            X = rearrange(X, 'n p d -> (n p) d').to(device)

            logits = model(X)
            loss = loss_fn(logits, Y)
            batch_loss = loss.item()
            total_val_loss += batch_loss

            preds = torch.argmax(logits, dim=1)
            correct_val += (preds == Y).sum().item()
            total_val += Y.size(0)

            batch_iou = compute_iou(preds.cpu().numpy(), Y.cpu().numpy(), config.num_classes)
            total_val_iou += batch_iou

            pbar.set_postfix(loss=f"{batch_loss:.2f}", iou=f"{batch_iou:.2f}")

    avg_val_loss = total_val_loss / len(test_dataloader)
    avg_val_accuracy = correct_val / total_val
    avg_val_iou = total_val_iou / len(test_dataloader)

    # ---------- LOGGING ----------
    wandb.log({
        "epoch": iter + 1,
        "train/loss": avg_train_loss,
        "train/accuracy": avg_train_accuracy,
        "train/iou": avg_train_iou,
        "val/loss": avg_val_loss,
        "val/accuracy": avg_val_accuracy,
        "val/iou": avg_val_iou
    })
    print()

    print(f"Epoch {iter+1}: "
          f"Train Loss={avg_train_loss:.4f}, Acc={avg_train_accuracy:.4f}, IoU={avg_train_iou:.4f} | "
          f"Val Loss={avg_val_loss:.4f}, Acc={avg_val_accuracy:.4f}, IoU={avg_val_iou:.4f}")

    print()

    # Save model checkpoint every 5 epochs
    if (iter + 1) % 5 == 0:
        checkpoint = {
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(), 
            'epoch': iter,
            'train_loss': avg_train_loss,
            'val_loss': avg_val_loss,
            'train_accuracy': avg_train_accuracy,
            'val_accuracy': avg_val_accuracy,
            'train_iou': avg_train_iou,
            'val_iou': avg_val_iou
        }
        
        torch.save(checkpoint, f'checkpoints/model_checkpoint_epoch_{iter+1}.pth')
        print(f"Saved model checkpoint to model_checkpoint_epoch_{iter+1}.pth")


wandb.finish()

Val Epoch 1/50: 100%|██████████| 4/4 [00:45<00:00, 11.37s/it, iou=0.04, loss=1.63]



Epoch 1: Train Loss=2.3683, Acc=0.4466, IoU=0.0308 | Val Loss=1.7224, Acc=0.5650, IoU=0.0334



Val Epoch 2/50: 100%|██████████| 4/4 [00:46<00:00, 11.54s/it, iou=0.05, loss=1.27]



Epoch 2: Train Loss=1.5737, Acc=0.5201, IoU=0.0414 | Val Loss=1.1276, Acc=0.5562, IoU=0.0506



Val Epoch 3/50: 100%|██████████| 4/4 [00:45<00:00, 11.45s/it, iou=0.08, loss=0.99]


...

Val Epoch 49/50: 100%|██████████| 4/4 [00:46<00:00, 11.63s/it, iou=0.19, loss=0.61]



Epoch 49: Train Loss=0.4698, Acc=0.8290, IoU=0.1418 | Val Loss=0.7153, Acc=0.7588, IoU=0.1229



Val Epoch 50/50: 100%|██████████| 4/4 [00:46<00:00, 11.51s/it, iou=0.18, loss=0.66]
wandb: ERROR The nbformat package was not found. It is required to save notebook history.



Epoch 50: Train Loss=0.4834, Acc=0.8253, IoU=0.1354 | Val Loss=0.7097, Acc=0.7581, IoU=0.1277

Saved model checkpoint to model_checkpoint_epoch_50.pth


epoch,▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
train/accuracy,▁▂▄▅▄▅▅▅▆▆▆▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇██████████
train/iou,▁▂▂▃▃▃▄▄▄▄▅▅▅▅▅▆▆▆▆▆▇▆▇▇▇▇▇▇▇▇▇█▇███▇█▇█
train/loss,█▅▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val/accuracy,▁▁▄▅▅▆▆▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇▇▇▇███▇████▇██████
val/iou,▁▂▃▄▄▅▅▅▅▆▆▆▆▆▆▆▆▇▇▇▇█▇█▇▇██▇██▇███▇████
val/loss,█▄▃▃▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch,50
train/accuracy,0.82526
train/iou,0.13539
train/loss,0.48342


In [ ]:
# checkpoint = {
#     'model_state_dict': model.state_dict(),
#     'optimizer_state_dict': optimizer.state_dict(), 
#     'epoch': iter,
#     'train_loss': avg_train_loss,
#     'val_loss': avg_val_loss,
#     'train_accuracy': avg_train_accuracy,
#     'val_accuracy': avg_val_accuracy,
#     'train_iou': avg_train_iou,
#     'val_iou': avg_val_iou
# }

# torch.save(checkpoint, f'checkpoints/model_checkpoint_epoch_{iter+1}.pth')
# print(f"Saved model checkpoint to model_checkpoint_epoch_{iter+1}.pth")
